# SPOR NW 2025: Exploring the Potential of Electronic Medical Records

## Load some made up discharge summaries

In [7]:
import pandas as pd

In [8]:
dsum = pd.read_csv("https://raw.githubusercontent.com/centre-for-health-informatics/SPORNW-2025-EMR-Workshop/refs/heads/master/synthetic_discharge_summaries.csv")
print(dsum.shape)
dsum.head(2)

(20, 2)


,title,summary
0,1. Acute Appendicitis,The patient is a 24-year-old male with no sign...
1,2. Pneumonia (Community-Acquired),The patient is a 67-year-old female with a his...


# Data Overview

# Determining disease status with Natural Language Processing (NLP)

## Keyword Search

## Regular Expressions
https://www.rexegg.com/regex-quickstart.php

## Negations

## Uncertainty

## Family history

## Extracting Values

## LLM